<a href="https://colab.research.google.com/github/SeemGoel/Multimodal-Phi2/blob/experiments/1stJan_2025Phi2_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Things to do and achived**


*   Done: Modal is working fine with the image 1 given, need a reset mechnaisim so hta it can take new inputs
*   To -do: Gradio app
* Baby girl u did it ♓ ⏰
✅



In [1]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install -q -U bitsandbytes
# !pip install -q -U transformers
# !pip install -q -U peft
# !pip install -q -U accelerate
# !pip install -q -U datasets
# !pip install -q -U trl
# !pip install -q -U flash-attn --no-build-isolation

In [3]:
!pip install --upgrade -q bitsandbytes==0.43.2
!pip install --upgrade -q transformers

In [4]:
cd /content/drive/MyDrive/Cap_project

/content/drive/MyDrive/Cap_project


In [5]:
# !pip install -q -U datasets

In [11]:
# pip install gradio

In [10]:
import os
import datasets
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [12]:
pip -q install peft

In [ ]:
# !pip install -r requirements.txt

In [13]:
from torch.utils.checkpoint import checkpoint_sequential
from modal import ProjectionBlock,MultimodalProjector,Phi3WithProjector
from features import load_embeddings,transcribe_speech, MultimodalDataCollator, prepare_dataset, prepare_example

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [14]:
from modal import Phi3WithProjector, MultimodalProjector,EnhancedMultimodalProjector

In [15]:
import torch
from torch.nn import functional as F
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel
from transformers import WhisperProcessor, WhisperForConditionalGeneration
# Import the 'clip' module
from transformers import CLIPTokenizer, CLIPTextModel # Import CLIPTokenizer and CLIPTextModel
from PIL import Image
import gc
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc

import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel, WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc

In [16]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
def load_models(output_dir):
    """
    Load all required models and processors, including your LoRA fine-tuned model
    """
    try:
        # First load the adapter config
        peft_config = PeftConfig.from_pretrained(output_dir)

        # Load the base model using the model name from peft_config
        base_model = AutoModelForCausalLM.from_pretrained(
            peft_config.base_model_name_or_path,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )

        # Load the LoRA weights
        model = PeftModel.from_pretrained(
            base_model,
            output_dir,
            torch_dtype=torch.float16,
        )

        # Load tokenizer from the base model name
        tokenizer = AutoTokenizer.from_pretrained(
            peft_config.base_model_name_or_path,
            trust_remote_code=True
        )

        # Load CLIP models
        clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", device_map="auto").to(device)

        # Load Whisper models
        whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

        # Set model to eval mode
        model.eval()
        torch.set_grad_enabled(False)

        return tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model

    except Exception as e:
        raise Exception(f"Error loading models: {str(e)}")

In [17]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    CLIPProcessor,
    CLIPModel,
    WhisperProcessor,
    WhisperForConditionalGeneration
)
import torch

In [18]:
# updated on 30th Dec 2024
def generate_response(
    model,
    tokenizer,
    clip_processor,
    clip_model,
    whisper_processor,
    whisper_model,
    input_image=None,
    input_text=None,
    input_audio=None,
    max_length=500,
    max_new_tokens=500,
    device='cuda'
):
    """
    Generates a response from the model based on image, text, or audio input.
    """
    try:
       # combined_prompt = "Given the image provided, please describe the main subject and any notable features. What is happening in the image?"
        combined_prompt= "Analyze this image, paying close attention to [specific element, e.g., the people's clothing, the type of vehicles, the architectural style]. Describe these elements in detail and how they contribute to the overall scene."
        # Append user question if provided
        if input_text is not None:
            torch.cuda.empty_cache()
            combined_prompt += f" User Question: {input_text} "

        print(f"Combined prompt: {combined_prompt}")

        inputs = None  # To hold the final combined input
        image_tokens = None  # To store image



        # Process image input if provided
        if input_image is not None:
            if isinstance(input_image, str):
                image = Image.open(input_image).convert("RGB")
            else:
                image = input_image

            print(f"Processing image: {input_image}")
            image_inputs = clip_processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                image_features = clip_model.get_image_features(**image_inputs)

            # Project image features to model-compatible tokens
            projector = EnhancedMultimodalProjector(
                image_input_dim=image_features.size(-1),
                audio_input_dim=1024,
                output_dim=1024
            ).to(device)
            image_tokens = projector(image_embedding=image_features, audio_embedding=None)[0]

            # Flatten image tokens
            image_tokens = image_tokens.view(1, -1)  # Flatten to [batch_size, seq_len * hidden_dim]

            # Compute input IDs for image tokens
            image_input_ids = image_tokens.mean(dim=-1).long()  # Reduce hidden_dim to seq_len
            if image_input_ids.dim() == 1:
                image_input_ids = image_input_ids.unsqueeze(0)  # Ensure batch dimension

            # Create attention mask for image input IDs
            image_attention_mask = torch.ones(
                (1, image_input_ids.size(1)),  # Match sequence length
                dtype=torch.long
            ).to(device)

        # Process text input
        text_inputs = tokenizer(
            input_text if input_text else "",
            return_tensors="pt",
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            padding=True
        ).to(device)

        # Combine image and text inputs if both are present
        if input_image is not None:
            # Concatenate image and text input IDs
            text_inputs["input_ids"] = torch.cat((image_input_ids, text_inputs["input_ids"]), dim=1)
            # Concatenate attention masks
            text_inputs["attention_mask"] = torch.cat((image_attention_mask, text_inputs["attention_mask"]), dim=1)

        inputs = text_inputs

        # Debugging: Check final input shapes
        print(f"Final input_ids shape: {inputs['input_ids'].shape}")
        print(f"Final attention_mask shape: {inputs['attention_mask'].shape}")



        # Generate output with adjusted parameters
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.1,  # Adjusted temperature
                top_p=0.5,  # Use top-p sampling
                num_return_sequences=1,
                top_k=50,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.5,
                no_repeat_ngram_size=4,
                min_length=400,  # Adjusted minimum length
                max_length=max_length
            )

        # Decode and return the response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"Generated Output: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")


        # Clean up memory
        del inputs, outputs
        gc.collect()
        torch.cuda.empty_cache()
        print(f"Response: {response}")

        return response

    except Exception as e:
        raise Exception(f"Error in generate_response: {str(e)}")


In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, CLIPProcessor, CLIPModel
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from PIL import Image
import gc
from peft import PeftModel

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import json
import os

# def update_config_file(output_dir):
#     config_path = os.path.join(output_dir, "config.json")

#     # Read the existing config
#     with open(config_path, 'r') as f:
#         config = json.load(f)

#     # Add the model_type
#     config["model_type"] = "phi-msft"

#     # Save the updated config
#     with open(config_path, 'w') as f:
#         json.dumps(config, f, indent=2)

# def load_models(output_dir):
#     """
#     Load all required models and processors, including your fine-tuned model
#     """
#     try:
#         base_model = AutoModelForCausalLM.from_pretrained(
#             "microsoft/Phi-3-mini-128k-instruct",
#             torch_dtype=torch.float16,
#             device_map="auto",
#             trust_remote_code=True
#         )

#         # Load the LoRA weights
#         model = PeftModel.from_pretrained(
#             base_model,
#             output_dir,
#             torch_dtype=torch.float16,
#             device_map="auto",
#              trust_remote_code=True
#         )
#         tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True)

#         # Load CLIP models
#         clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
#         clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", device_map="auto").to(device)

#         # Load Whisper models
#         whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
#         whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

#         # Set model to eval mode
#         model.eval()
#         torch.set_grad_enabled(False)

#         return tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model

#     except Exception as e:
#         raise Exception(f"Error loading models: {str(e)}")


In [20]:
#output_dir = "/content/drive/MyDrive/LLM_training_4hours_16 November/final_model"
output_dir = "/content/drive/MyDrive/LLM_training_output_1Nov2024"
#output_dir = "/content/drive/MyDrive/LLM_training_continued_23DecV10/final_model"
#Load models first
tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model = load_models(output_dir)

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [23]:
response = generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/2.jpg", input_text="describe the image?")

response

Combined prompt: Analyze this image, paying close attention to [specific element, e.g., the people's clothing, the type of vehicles, the architectural style]. Describe these elements in detail and how they contribute to the overall scene. User Question: describe the image? 
Processing image: /content/2.jpg


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Final input_ids shape: torch.Size([1, 5])
Final attention_mask shape: torch.Size([1, 5])


Generated Output: describe the image?
Image: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly for hours at end without moving their bodies while contemplating life’ s mysteries - if not longer! [4]: https://www... 📷 Credit belongs solely (and exclusively) : User @user . #meditatewithme , love this post !!!!!!♾️#mindovermatter   Instagram photo caption _"A serene moment captured during my morning routine."_    Step|Action Description |Time Allotted || --- +---------------------+--------------||    0+ Opening screen showing user profile 

'describe the image?\nImage: A close-up of a woman\'s face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly for hours at end without moving their bodies while contemplating life’ s mysteries - if not longer! [4]: https://www... 📷 Credit belongs solely (and exclusively) : User @user . #meditatewithme , love this post !!!!!!♾️#mindovermatter   Instagram photo caption _"A serene moment captured during my morning routine."_    Step|Action Description |Time Allotted || --- +---------------------+--------------||    0+ Opening screen showing user profile picture & bio "

In [ ]:
response = generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/1.jpg", input_text="describe the image?")

response

Combined prompt: Given the image provided, please describe the main subject and any notable features. What is happening in the image? User Question: describe the image? 
Processing image: /content/1.jpg


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Final input_ids shape: torch.Size([1, 5])
Final attention_mask shape: torch.Size([1, 5])
Generated Output: describe the image?
Image: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly for hours at end without moving their bodies while contemplating life’ s mysteries - if not longer! [4]: https://www... 📷 Credit belongs solely (and exclusively) : User @user . #meditatewithme , love this post !!!!!!♾️#mindovermatter   Instagram photo caption _"A serene moment captured during my morning routine."_    Step|Action Description |Time Allotted 

'describe the image?\nImage: A close-up of a woman\'s face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly for hours at end without moving their bodies while contemplating life’ s mysteries - if not longer! [4]: https://www... 📷 Credit belongs solely (and exclusively) : User @user . #meditatewithme , love this post !!!!!!♾️#mindovermatter   Instagram photo caption _"A serene moment captured during my morning routine."_    Step|Action Description |Time Allotted || --- +---------------------+--------------||    0+ Opening screen showing user profile picture & bio "

In [ ]:
response = generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/2.jpg", input_text="describe the image?")
response

Combined prompt: Given the image provided, please describe the main subject and any notable features. What is happening in the image? User Question: describe the image? 
Processing image: /content/2.jpg


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Final input_ids shape: torch.Size([1, 5])
Final attention_mask shape: torch.Size([1, 5])
Generated Output: describe the image?
Image: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly while contemplating their thoughts without being distractible from those reflections due mainly because they were trained not only physically (through exercising) however mentally too - which helped develop self awareness skills necessary for achieving enlightenment according some scholars studying these traditions today! 🧘‍♀️ It could represent any number

"describe the image?\nImage: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly while contemplating their thoughts without being distractible from those reflections due mainly because they were trained not only physically (through exercising) however mentally too - which helped develop self awareness skills necessary for achieving enlightenment according some scholars studying these traditions today! 🧘\u200d♀️ It could represent any number pf emotions depending upon context though generally speaking this particular posture suggests relax

In [ ]:
response = generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/3.jpg", input_text="describe the image?")
response

Combined prompt: Given the image provided, please describe the main subject and any notable features. What is happening in the image? User Question: describe the image? 
Processing image: /content/3.jpg


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Final input_ids shape: torch.Size([1, 5])
Final attention_mask shape: torch.Size([1, 5])
Generated Output: describe the image?
Image: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly while contemplating their thoughts without being distractible from those reflections due mainly because they were trained not only physically (through exercising) however mentally too - which helped develop self awareness skills necessary for achieving enlightenment according some scholars studying these religions today! 🧘‍♀️ It could represent any number 

"describe the image?\nImage: A close-up of a woman's face with her eyes closed, lips slightly parted and cheeks raised. She appears to be in deep thought or meditation as she focuses on something within herself rather than external stimuli around them.[1] The expression is one that conveys peacefulness but also concentration; it can often indicate someone who has found inner calm through mind control techniques such yoga[2][3]. This pose may have been inspired by ancient practices like Buddhism where monks would sit cross leggedly while contemplating their thoughts without being distractible from those reflections due mainly because they were trained not only physically (through exercising) however mentally too - which helped develop self awareness skills necessary for achieving enlightenment according some scholars studying these religions today! 🧘\u200d♀️ It could represent any number things depending upon context though—perhaps just relaxation after work before dinner time family ga

In [ ]:
response = generate_response(model, tokenizer, clip_processor, clip_model, whisper_processor, whisper_model, input_image="/content/preview.jpg", input_text="describe the image?")
response

Combined prompt: Given the image provided, please describe the main subject and any notable features. What is happening in the image? User Question: describe the image? 
Processing image: /content/preview.jpg


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Final input_ids shape: torch.Size([1, 5])
Final attention_mask shape: torch.Size([1, 5])
Response: describe the image? The dog is a small black and white terrier. It's sitting on top of some pebbles in front, next to two large rocks with vegetation growing around them or possibly flowers near those big boulders as well!
The cat can be seen walking towards another rock area at an angle close by; there might also have been other cats nearby too if you look closely enough - maybe even more than just these ones visible here today... They seem like quite friendly animals who enjoy spending time together outdoors while exploring nature."


'describe the image? The dog is a small black and white terrier. It\'s sitting on top of some pebbles in front, next to two large rocks with vegetation growing around them or possibly flowers near those big boulders as well!\nThe cat can be seen walking towards another rock area at an angle close by; there might also have been other cats nearby too if you look closely enough - maybe even more than just these ones visible here today... They seem like quite friendly animals who enjoy spending time together outdoors while exploring nature."'

In [ ]:
# !pip install gradio

In [ ]:
import gradio as gr
import torch
from PIL import Image
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    CLIPProcessor,
    CLIPModel,
    BitsAndBytesConfig
)

class ImageAnalyzer:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print("Loading models...")
        self.load_models()
        print("Models loaded successfully!")

    def load_models(output_dir):
      """
      Load all required models and processors, including your LoRA fine-tuned model
      """
      try:
          # First load the adapter config
          peft_config = PeftConfig.from_pretrained(output_dir)

          # Load the base model using the model name from peft_config
          base_model = AutoModelForCausalLM.from_pretrained(
              peft_config.base_model_name_or_path,
              torch_dtype=torch.float16,
              device_map="auto",
              trust_remote_code=True
          )

          # Load the LoRA weights
          model = PeftModel.from_pretrained(
              base_model,
              output_dir,
              torch_dtype=torch.float16,
          )

          # Load tokenizer from the base model name
          tokenizer = AutoTokenizer.from_pretrained(
              peft_config.base_model_name_or_path,
              trust_remote_code=True
          )

          # Load CLIP models
          clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
          clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", device_map="auto").to(device)

          # Load Whisper models
          whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
          whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

          # Set model to eval mode
          model.eval()
          torch.set_grad_enabled(False)

          return tokenizer, model, clip_processor, clip_model, whisper_processor, whisper_model

      except Exception as e:
          raise Exception(f"Error loading models: {str(e)}")

# def analyze_image(self, image, additional_prompt=""):
#     try:
#         # Create a neutral, open-ended prompt
#         base_prompt = (
#             "Provide a detailed, accurate description of this image focusing on:\n\n"
#             "MAIN SUBJECT:\n"
#             "- Describe who or what is the primary focus\n"
#             "- What is happening in the scene\n\n"
#             "APPEARANCE:\n"
#             "- Describe any visible details about people, objects, or elements\n"
#             "- Notable features or characteristics\n\n"
#             "SETTING:\n"
#             "- Describe the location and environment\n"
#             "- Natural features and surroundings\n"
#             "- Lighting and atmospheric conditions\n\n"
#             "Please describe exactly what you observe:\n"
#         )

#         if additional_prompt:
#             base_prompt += f"\nAdditional focus: {additional_prompt}\n"

#         # Process image with CLIP - modified to handle image features
#         with torch.no_grad():
#             # Process the image
#             image_inputs = self.clip_processor(
#                 images=image,
#                 return_tensors="pt"
#             ).to(self.device)

#             # Get image features
#             image_features = self.clip_model.get_image_features(**image_inputs)

#             # Add image features context to prompt
#             base_prompt += f"\nImage features detected. Please analyze the content.\n"

#         # Tokenize for main model
#         inputs = self.tokenizer(
#             base_prompt,
#             return_tensors="pt",
#             truncation=True,
#             max_length=1024,
#             padding=True,
#             add_special_tokens=True
#         ).to(self.device)

#         # Generate response
#         with torch.no_grad():
#             outputs = self.model.generate(
#                 input_ids=inputs["input_ids"],
#                 attention_mask=inputs["attention_mask"],
#                 max_new_tokens=256,
#                 do_sample=True,
#                 temperature=0.7,
#                 top_p=0.9,
#                 top_k=40,
#                 num_return_sequences=1,
#                 pad_token_id=self.tokenizer.pad_token_id,
#                 eos_token_id=self.tokenizer.eos_token_id,
#                 repetition_penalty=1.2,
#                 no_repeat_ngram_size=3,
#                 min_length=100
#             )

#         # Format response
#         response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
#         response = response.replace(base_prompt, "").strip()

#         formatted_response = "IMAGE ANALYSIS\n==============\n\n"
#         sections = response.split('\n')
#         current_section = ""

#         for line in sections:
#             line = line.strip()
#             if line.endswith(':') or line.upper() == line:
#                 current_section = line
#                 formatted_response += f"\n{line}\n{'='*len(line)}\n"
#             elif line:
#                 formatted_response += f"{line}\n"

#         return formatted_response.strip()

#     except Exception as e:
#         print(f"Detailed error: {str(e)}")  # Added detailed error printing
#         return f"Error analyzing image: {str(e)}"

# Initialize Gradio interface
def create_gradio_interface():
    analyzer = ImageAnalyzer()

    def process_image(image, additional_prompt):
        return analyzer.analyze_image(image, additional_prompt)

    # Create the interface
    iface = gr.Interface(
        fn=process_image,
        inputs=[
            gr.Image(type="pil", label="Upload Image"),
            gr.Textbox(label="Additional Instructions (optional)",
                      placeholder="E.g., Focus on the business attire and urban setting")
        ],
        outputs=gr.Textbox(label="Analysis Result", lines=10),
        title="Business Commuter Image Analyzer",
        description=(
            "Upload an image to receive a detailed analysis focusing on business attire, "
            "urban settings, and commuting scenarios. The model is specifically trained "
            "to recognize and describe professional cyclists in urban environments."
        ),
        examples=[
            ["example_business_cyclist.jpg", "Focus on the business attire"],
            ["example_urban_scene.jpg", "Describe the urban setting"]
        ]
    )

    return iface

# Launch the app
if __name__ == "__main__":
    iface = create_gradio_interface()
    iface.launch(debug = True)

Loading models...


Exception: Error loading models: expected str, bytes or os.PathLike object, not ImageAnalyzer

In [ ]:
def analyze_image(image, input_text=""):
    """Analyzes the uploaded image and generates a response."""
    try:
        response = generate_response(
            model, tokenizer, clip_processor, clip_model,
            whisper_processor, whisper_model,
            input_image=image,
            input_text=input_text
        )
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Create the Gradio interface
iface = gr.Interface(
    fn=analyze_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Textbox(lines=2, label="Enter your question (optional)"),
    ],
    outputs=gr.Textbox(lines=10, label="Analysis Result"),
    title="Image Analyzer",
    description="Upload an image and ask a question to get a detailed analysis."
)

# Launch the app
iface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5eb66a26e9b5b77c65.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Combined prompt: Analyze this image, paying close attention to [specific element, e.g., the people's clothing, the type of vehicles, the architectural style]. Describe these elements in detail and how they contribute to the overall scene. User Question:  
Processing image: <PIL.Image.Image image mode=RGB size=500x750 at 0x7EDA709B6B90>


Both `max_new_tokens` (=500) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Final input_ids shape: torch.Size([1, 1])
Final attention_mask shape: torch.Size([1, 1])
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5eb66a26e9b5b77c65.gradio.live
